## Script for Developing Phase One Imagery
### developPhaseOne_v2.0.py
### J. Heath Harwood

In [16]:
import sys
sys.version

'3.7.4 (default, Aug 13 2019, 20:35:49) \n[GCC 7.3.0]'

In [17]:
import os,re,sys,easygui,time,shutil
import platform
from datetime import datetime
import phaseoneimageproc as poip
import folium
import json
import calEO
import easygui

In [18]:
pyFileVersion = 'developPhaseOne.py v 2.0'

In [19]:
print ("This is the scripts directory where developPhaseOne.py is running >>> " + os.getcwd())
print ("This is the scripts is running >>> " + pyFileVersion)

This is the scripts directory where developPhaseOne.py is running >>> /home/jovyan/work
This is the scripts is running >>> developPhaseOne.py v 2.0


### Specify Input Directoryies using GUI

In [2]:
''' This is a GUI layout using PySimpleGUI
    The purpose of this gui is to:
        - define file paths to processing raw image directories
        - define the zone number and zone hemisphere
        - get the number of processors for running COPE and IrfanView

'''
import PySimpleGUI as sg
# Defining the column layout for number of processors to COPE to use; we also define it's value key in order to
# load the number or procs to a variable
column1 = [
            [sg.Text('COPE', background_color='#FFFFFF', justification='center', size=(10, 1))],      
            [sg.Spin([i for i in range(1,13)], initial_value=4,key='_nCopeProc_',tooltip='Best to keep the tif generation to 4 CPUs'), sg.Text('Number of COPE CPUs')],            
        ] 

# Defining the column layout for number of processors to IrfanView to use; we also define it's value key in order to
# load the number or procs to a variable
column2 = [
            [sg.Text('IRFANVIEW', background_color='#FFFFFF', justification='center', size=(10, 1))],      
            [sg.Spin([i for i in range(1,13)], initial_value=4,key='_nIrfanProc_',tooltip='Best to keep the jpeg generation to 4 CPUs'), sg.Text('Number of irFanView CPUs')],            
        ] 

# Define the layout of the entire GUI here with value keys in order to assign selections to variables
layout = [ 
            [sg.Text('Flight Date Block Folder',size=(18, 1)),sg.InputText(key='_block1_',tooltip='Browse to Block Folder Only!'), sg.FolderBrowse(initial_folder=r'D:\2019_NCMP_GL\processing\camera')],
            [sg.Text('Flight Date Block Folder',size=(18, 1)),sg.InputText(key='_block2_',tooltip='Browse to Block Folder Only!'), sg.FolderBrowse(initial_folder=r'D:\2019_NCMP_GL\processing\camera')],
            [sg.Text('Flight Date Block Folder',size=(18, 1)),sg.InputText(key='_block3_',tooltip='Browse to Block Folder Only!'), sg.FolderBrowse(initial_folder=r'D:\2019_NCMP_GL\processing\camera')],
            [sg.Text('Flight Date Block Folder',size=(18, 1)),sg.InputText(key='_block4_',tooltip='Browse to Block Folder Only!'), sg.FolderBrowse(initial_folder=r'D:\2019_NCMP_GL\processing\camera')],
            [sg.Text('Flight Date Block Folder',size=(18, 1)),sg.InputText(key='_block5_',tooltip='Browse to Block Folder Only!'), sg.FolderBrowse(initial_folder=r'D:\2019_NCMP_GL\processing\camera')],
            [sg.Text('Flight Date Block Folder',size=(18, 1)),sg.InputText(key='_block6_',tooltip='Browse to Block Folder Only!'), sg.FolderBrowse(initial_folder='D:\\')],
            [sg.Text('Flight Date Block Folder',size=(18, 1)),sg.InputText(key='_block7_',tooltip='Browse to Block Folder Only!'), sg.FolderBrowse(initial_folder='D:\\')],
            [sg.Text('Flight Date Block Folder',size=(18, 1)),sg.InputText(key='_block8_',tooltip='Browse to Block Folder Only!'), sg.FolderBrowse(initial_folder='D:\\')],
            [sg.Text('Flight Date Block Folder',size=(18, 1)),sg.InputText(key='_block9_',tooltip='Browse to Block Folder Only!'), sg.FolderBrowse(initial_folder='D:\\')],
            [sg.Text('Zone Number', size=(15, 1)), sg.InputText('16', key='_zoneNum_',tooltip='For > 50% of data')],
            [sg.Text('Zone Hemisphere', size=(15, 1)), sg.InputText('N', key='_zoneHem_',tooltip='N Unless South of Equator')],
            [sg.Text('Camera and Install', size=(18, 1)),sg.InputCombo(['YC030333_20200113','MM000134_20190905','MM000174_20190607','YC030284_20190514','MM000174_20190325','YC030333_20181115'],key='_caminstall_',pad=(5,5),size=(22, 1),tooltip='Check the spreadsheet')],
            [sg.Text('Support Folder Path',size=(18, 1)),sg.InputText(r'B:\Support',key='_supportpath_',tooltip='Set this path if not correct'), sg.FolderBrowse(initial_folder=r'B:\Support')],
            [sg.Frame('Number of Processors',[[sg.Column(column1, background_color='#bebebe'),
             sg.Column(column2, background_color='#bebebe')]])],
            [sg.Frame('Develop both the PhaseOne Images tifs and scaled jpegs?',[[sg.InputCombo(['Tifs and Scaled Jpegs','Jpgs and Scaled Jpegs','Tifs Only', 'Scaled Jpegs Only','Rerun the EO/KMZ'],
                                                             key='_choice_',pad=(10,10))]])],
            [sg.Frame('Sbet bet available?', [[sg.Checkbox('Have Sbet', default=True,key='checked',pad=(5,5),tooltip="Uncheck if an sbet hasn't been created yet." )]])],
            [sg.Text('Path to COPE executable',size=(21, 1)),sg.InputText(r'C:\cope\cope.exe',key='_cope_',tooltip='Set this path if not correct'), sg.FolderBrowse(initial_folder=r'C:\cope')],
            [sg.Text('Path to IrFanView executable',size=(21, 1)),sg.InputText(r'C:\IrfanView\i_view64.exe',key='_irfanview_',tooltip='Set this path if not correct'), sg.FolderBrowse(initial_folder=r'C:\IrfanView')],
            [sg.Submit(),sg.Exit()]

        ]
           
screenlocation = (600,200)

window = sg.Window('Phase One Flight Date Processing', location=screenlocation).Layout(layout)

event, values = window.Read()   
    
print (values['_block1_'],values['_block2_'],values['_block3_'],\
values['_block4_'],values['_block5_'],values['_block5_'],values['_block6_'],\
values['_block7_'],values['_block8_'],values['_block9_'],values['_zoneNum_'],values['_zoneHem_'],\
values['_nCopeProc_'],values['_nIrfanProc_'],values['_choice_'],values['_cope_'],values['_irfanview_'],\
values['_caminstall_'],values['_supportpath_'],values['checked'])

window.Close()


TclError: no display name and no $DISPLAY environment variable

In [ ]:
zoneNum = values['_zoneNum_']
zoneHem = values['_zoneHem_']
#print zoneNum, zoneHem

### Define number of processors

In [ ]:
nCopeProc = int(values['_nCopeProc_'])
cope = values['_cope_']
nIrfanProc = int(values['_nIrfanProc_'])
irfanview = values['_irfanview_']
#print nCopeProc, nIrfanProc
copeSwCmd = []
irfSwCmd = []
prev_time = -1

### Scan Block Folders to get images and exif logs

In [ ]:
blocks = [values['_block1_'],values['_block2_'],\
          values['_block3_'],values['_block4_'],\
          values['_block5_'],values['_block6_'],\
          values['_block7_'],values['_block8_'],\
          values['_block9_']]
blocks = list(filter(None,blocks))

camInstall = values['_caminstall_']
supportPath = values['_supportpath_']

# Loop through raw block folders and process data
for block in blocks:
    
    # Scan block folders and get block info (ie, raw block path, block name, 
    # area name, exif log, camera system model number)
    blockInfo = poip.scanBlock(block)
    rawBlkPath = blockInfo[0]
    #print rawBlkPath
    blkName = blockInfo[1]
    areaName = blockInfo[2][0]
    flightDateNum = blockInfo[3][0]
    exifLog = blockInfo[4]
    camSysModel = blockInfo[5][0]

    sbetProc = values['checked']
    path2Imgs = os.path.join(str(rawBlkPath + '\\' + camSysModel))
    path2exifLog = os.path.join(str(rawBlkPath + '\\' + camSysModel + '\\' + exifLog))
    
    
    # Get the first event time in the exif log to create the DC directory time stamp
    evt1TimeName = poip.getEvt1Time(path2exifLog)
    
    # Set the flight date and timestamp for the DC directory
    flightDate = flightDateNum[2:8]
    dcTimeStamp = evt1TimeName[0] + evt1TimeName[1] 
    # DC directory name will only have the date and timestamp
    # Letter can be appended when Lidar dataset is known
    dcDirName = ("DC_DS_P_%s_%s")%(flightDate,dcTimeStamp)
    driveLetter = rawBlkPath.split('\\')[0]
    camDir = rawBlkPath.split('\\')[1:5]
    camAreaDir = os.path.join(*camDir)
    datasetLoc = driveLetter + '\\' + camAreaDir
    
    # Build Block directory
    blkDir = datasetLoc + '\\' + blkName
    poip.buildBlkDir(blkDir)

    # Define and build dataset directory
    tifPath = datasetLoc + '\\' + blkName + '\\' + dcDirName
    poip.buildDSDir(tifPath)
    
    # Create log file
    logTimeName = datetime.now().strftime(tifPath+'\\'+dcDirName+'_proc_log_%Y%m%d_%H%M%d.txt')
    logFile =open(logTimeName, 'w')

    # set start timer to gage processing time
    start = time.time()
    startTimeMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Starting PhaseOne processing script.\n"
    logFile.writelines(startTimeMsg)
    print (startTimeMsg)

    # Get computer name
    compName = platform.uname()[1]
    compNameInfoMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Data is being processed on "+compName+".\n"
    logFile.writelines(compNameInfoMsg)
    print (compNameInfoMsg)

    # print the file name in the log file
    pyFileRunMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Data is being processed with "+pyFileVersion+".\n"
    logFile.writelines(pyFileRunMsg)
    print (pyFileRunMsg)

    # Define year/month/day
    year = "20" + str(dcDirName[8:10])
    month = str(dcDirName[10:12])
    day = str(dcDirName[12:14])
    #print year, month, day
    # Name the kml from the name of the common working directory "DC_*"
    name2 = str(dcDirName)
    
    # Copy czmil system files in to DC folder for IPAS processing
    poip.copySysFiles(camInstall,tifPath,supportPath)

    # Create the output files
    evtFile = open(tifPath+'\\'+'event.evt', 'w')
    frameSer = open(tifPath+'\\'+'frame_serialnumber.ser', 'w')
    photoID = open(tifPath+'\\'+'photoID.txt', 'w')
    photoIDPix4D = open(tifPath+'\\'+'photoID_for_pix4d.txt', 'w')
    coarseDat = open(tifPath+'\\'+'coarse_lat_lon_ellHeight_roll_pitch_heading.dat', 'w')

    # Create the master kml file in the mission directory and open for writing.
    print "Generating a KMZ file named: \n" + '_' +name2+"_thumbs.kmz\n"
    kml = open(os.path.join(tifPath+'\\'+'_' + name2+ "_thumbs.kml"), 'w')
    #print kml

    # Create the camera sync file needed for pfm3D Editor to view images during editing
    csfName = str(dcDirName[8:])
    cameraSyncR = open((tifPath+'\\'+'CameraSync_' + csfName + '_A_R.dat'), 'w')
    #print cameraSyncR

    # System number is used for chartsPic is actually not a CZMIL system but instead not a hof/tof/CM4800 file and disgnates the RCD30/PhaseOne Camera
    sysNumR = ' system_num 4\n'
    cameraSyncR.writelines(sysNumR)
    
    # Parse ExifLog to create IPAS Files
    # print the file name in the log file
    exifDataInfoMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d') + "    Parsing ExifLog.csv.\n"
    logFile.writelines(exifDataInfoMsg)
    print (exifDataInfoMsg)

    # Get EO data 
    eoData = poip.readExif(path2exifLog)
    
    # Create ipas files 
    poip.createIPAS(eoData, year, month, day, evtFile, photoID, photoIDPix4D, frameSer, coarseDat) 
    
    # Create the KML/KMZ from exif data
    # Start looping through dat Check for text to skip header.
    # print the file name in the log file
    kmlLoopOverlayInfoMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Starting KML Overlay Loop...\n"
    logFile.writelines(kmlLoopOverlayInfoMsg)
    print (kmlLoopOverlayInfoMsg)

    poip.processKML(eoData, name2, kml, year, month, day, prev_time, zoneNum, zoneHem, cameraSyncR)
    
    # Create the GeoJSON from exif data
    location = poip.createGeoJson(eoData, tifPath, dcDirName)
    geoLocLat = location[0]
    geoLocLon = location[1]
   
    geo_path = tifPath+'\\'+dcDirName+'.geojson'
    print (geo_path)

    # Link to Esri World Imagery service plus attribution
    EsriImagery = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
    EsriAttribution = "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"

    m = folium.Map(location=[geoLocLat,geoLocLon], tiles=EsriImagery, attr=EsriAttribution, zoom_start=12)
    folium.GeoJson(geo_path,name=name2,tooltip=folium.GeoJsonTooltip(fields=['Filename','Latitude','Longitude','GPS Event','GPS Time','Weeks:Seconds','Pitch','Roll','Yaw'])).add_to(m)
    folium.LayerControl().add_to(m)
    # Load map to notebook
    m
    # save map to notebook
    geohtml = tifPath+'\\'+dcDirName+'_geojson_events.html'
    folium.Map.save(m, geohtml)
    
    # Get number of images from image directory
    
    # Call function to get number of images
    totFiles = poip.getNumImgs(path2Imgs)

    # Write state to log
    totFilesInfoMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Total number of images "+str(totFiles)+".\n"
    logFile.writelines(totFilesInfoMsg)
    print (totFilesInfoMsg)
    
    # Start Image processing
    # Write state to log
    tifJpegFilesInfoMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Started Image Processing.\n"
    logFile.writelines(tifJpegFilesInfoMsg)
    print (tifJpegFilesInfoMsg)

    # define choice and call processImgs function
    choice = values['_choice_']
    poip.processImgs(poip.getImgs(path2Imgs),tifPath,copeSwCmd,irfSwCmd,nCopeProc,\
                     nIrfanProc,kml,name2,cameraSyncR,evtFile,photoID,frameSer,choice,cope,irfanview)

    tifFinishedInfoMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Finshed Image Processing.\n"
    logFile.writelines(tifFinishedInfoMsg)
    print (tifFinishedInfoMsg)

    # Write state to log
    totTime = (time.time() - start)/60 
    TotTimeInfoMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Total time to process data "+str(totTime)+" minutes.\n"
    logFile.writelines(TotTimeInfoMsg)
    print (TotTimeInfoMsg)

    # Write state to log
    secperimg =(time.time() - start)/totFiles
    secpreimgTimeInfoMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Processed time per image "+str(secperimg)+" seconds.\n"
    logFile.writelines(secpreimgTimeInfoMsg)
    print (secpreimgTimeInfoMsg)
    
    if sbetProc == True:
        sbetFile = blockInfo[6]
        path2sbetFile = os.path.join(str(rawBlkPath + '\\' + camSysModel + '\\' + sbetFile))
        path2sbetFileDC = tifPath + '\\' + sbetFile
        shutil.copy2(path2sbetFile, path2sbetFileDC)
        # Process the sbet file and create an EO file
        
        # Write state to log
        eoStartInfoMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Started EO Processing.\n"
        logFile.writelines(eoStartInfoMsg)
        print (eoStartInfoMsg)
        
        # Run the EO calculator
        calEO.calEO(eoData, path2sbetFile, tifPath, dcDirName, int(zoneNum))
        
        # Write state to log
        eoFinishedInfoMsg = datetime.now().strftime('%m\%d\%Y %H:%M:%d')+"    Finshed EO Processing.\n"
        logFile.writelines(eoFinishedInfoMsg)
        print (eoFinishedInfoMsg)
        
    else:
        continue
    
    # Clean up
    logFile.flush()
    logFile.close()

    del start

    sleepsec = 1
    print ("Finished, closing in:")
    print ("1 Seconds")
    time.sleep(sleepsec)